In [1]:
pip install boto3


In [2]:
import os
import zipfile
import csv
from datetime import datetime
import boto3
from io import StringIO
import pandas as pd

In [4]:
## set-up for AWS  
s3_client = boto3.client('s3')  
#s3_cl = boto3.resource('s3')  
bucket_name = 'ca-climate-index'  
directory = '1_pull_data/climate_risk/extreme_heat/loss/usda/usda_crop_loss_heat_files/'  

C:\Users\jesse\.aws\credentials


ConfigParseError: Unable to parse config file: C:\Users\jesse/.aws/credentials

In [ ]:
# Function to extract and read text files from zip files
def extract_and_read(file_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        file_contents = []
        for file in file_list:
            with zip_ref.open(file) as txt_file:
                lines = txt_file.readlines()
                lines = [line.decode('utf-8').strip() for line in lines]
                file_contents.append((file, lines))
    return file_contents

In [ ]:
# Function to extract year from line
def get_year_from_line(line):
    # Year is separated by '|' and is the first element
    elements = line.split('|')
    if len(elements) > 1:
        potential_year = elements[0].strip()
        if len(potential_year) == 4 and potential_year.isdigit():
            return potential_year
    return ''

In [ ]:
# Function to sort files based on the first column
def sort_files(file_contents):
    sorted_files = []
    for file, lines in file_contents:
        lines.sort(key=lambda x: get_year_from_line(x))
        sorted_files.append((file, lines))
    return sorted_files

In [ ]:
# Modify the merge_files function to use the client for S3 operations
def merge_files(sorted_files, output_file):
    headers = ['year', 'state_code', 'state_abbreviation', 'county_code',
               'county_name', 'commodity_code', 'commodity_name',
               'insurance_plan_code', 'insurance_plan_abbreviation',
               'stage_code', 'damage_cause_code', 'damage_description',
               'determined_acres', 'indemnity_amount']

    # Create an empty list to hold rows
    rows = []

    for file, lines in sorted_files:
        for line in lines:
            row_data = [element.strip() for element in line.split('|')]
            if len(row_data) > 2 and row_data[2] == 'CA':
                rows.append(row_data)

    # Create DataFrame from the list of rows
    df = pd.DataFrame(rows, columns=headers)

    # Save DataFrame to AWS using the client
    new_buffer = StringIO()
    df.to_csv(new_buffer, index=False)
    content = new_buffer.getvalue()
    s3_client.put_object(Bucket=bucket_name, Body=content, Key=directory + output_file)

    # Optionally, return the DataFrame
    return df

In [ ]:
# Main function
def sort_and_merge(folder_name, output_file):
    '''
    Extracts data from multiple files within a folder, assigns each column an appropriate header, merges all
    datasets together, sorts by data year, and writes the final output as a .csv file
    
    Parameters
    ----------
    folder_name: string
              The name of the folder containing zipped files for 'Cause of Loss Historical Data Files' under 'Indemnities Only'
              from the USDA's Risk Management Agency: https://legacy.rma.usda.gov/data/cause.html
    output_file: string
              Final output as a .csv file
    '''
    file_contents = []
    for file in os.listdir(folder_name):
        if file.endswith('.zip'):
            file_path = os.path.join(folder_name, file)
            file_contents.extend(extract_and_read(file_path))

    sorted_files = sort_files(file_contents)
    final_df = merge_files(sorted_files, output_file)
    print(f"Merged and sorted files written to {output_file}")

    # Return the final DataFrame if needed
    return final_df

In [ ]:
# Provide folder path containing zip files and output file name
# Zip files were downloaded and placed in a folder within our local env
folder_name = 'usda_crop_loss_heat_files'
output_file = 'usda_crop_loss_CA_boto.csv'

# Run the main function
sort_and_merge(folder_name, output_file)